In [ ]:
from model_class import *
from modelUtils import *
from config import Config
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [ ]:
## Gather and clean data

import pickle
with open('data_lr.pkl', 'rb') as file:
    dataset, personID, speedID = pickle.load(file)
print(len(dataset), len(personID), len(speedID))


In [ ]:
N_SELECT = 100
batch_size = 16
num_workers = 4
N_P_CLASS_KTEST = 4

In [ ]:
### Train open set user identification model
for i in range(1):
    (x_train, y1_train, y2_train), (x_ktest, y1_ktest, y2_ktest), (x_utest, y1_utest, y2_utest) = split_dataset(dataset, personID, speedID)
    (x_train_small, y1_train_small, y2_train_small), (x_train_large, y1_train_large, y2_train_large) = split_small_large_set(x_train, y1_train, y2_train, N_SELECT)
    (x_ktest_small, y1_ktest_small, y2_ktest_small), (x_ktest_large, y1_ktest_large, y2_ktest_large) = split_small_large_set(x_ktest, y1_ktest, y2_ktest, N_SELECT)
    (x_utest_small, y1_utest_small, y2_utest_small), (x_utest_large, y1_utest_large, y2_utest_large) = split_small_large_set(x_utest, y1_utest, y2_utest, 200)
    x_ktest_large_ = torch.cat((x_ktest_large, x_utest_large), dim = 0)
    y1_ktest_large_ = torch.cat((y1_ktest_large, torch.full(y1_utest_large.shape, 4)), dim = 0)
    
    opt = Config()
    NUM_CLASSES = 7
    
    torch.cuda.empty_cache()
    models = Ensemble()
    print('Training started')
    file_name = 'model_user'
    data = model_train_eval(model, opt, NUM_CLASSES, file_name, x_train_large, y1_train_large, x_ktest_small, y1_ktest_small, x_ktest_large_, y1_ktest_large_)
    print('Training ended')

In [ ]:
### Train speed identification model
for i in range(1):
    (x_train, y1_train, y2_train), (x_ktest, y1_ktest, y2_ktest), (x_utest, y1_utest, y2_utest) = split_dataset(dataset, personID, speedID)
    (x_train_small, y1_train_small, y2_train_small), (x_train_large, y1_train_large, y2_train_large) = split_small_large_set(x_train, y1_train, y2_train, N_SELECT)
    (x_ktest_small, y1_ktest_small, y2_ktest_small), (x_ktest_large, y1_ktest_large, y2_ktest_large) = split_small_large_set(x_ktest, y1_ktest, y2_ktest, N_SELECT)
    (x_utest_small, y1_utest_small, y2_utest_small), (x_utest_large, y1_utest_large, y2_utest_large) = split_small_large_set(x_utest, y1_utest, y2_utest, 200)
    x_ktest_large_ = torch.cat((x_ktest_large, x_utest_large), dim = 0)
    y1_ktest_large_ = torch.cat((y1_ktest_large, torch.full(y1_utest_large.shape, 4)), dim = 0)
    
    opt = Config()
    NUM_CLASSES = 5
    
    torch.cuda.empty_cache()
    models = Ensemble()
    print('Training started')
    file_name = 'model_speed'
    data = model_train_eval(model, opt, NUM_CLASSES, file_name, x_train_large, y2_train_large, x_ktest_small, y2_ktest_small, x_ktest_large_, y2_ktest_large_)
    print('Training ended')

In [23]:
### Import saved user model
import pickle
import torch
with open('ModelN/model_user.pkl','rb') as f:
    data = pickle.load(f)
model = data[0]
model.eval()
    

In [ ]:
### Evaluate saved model
(x_train, y1_train, y2_train), (x_ktest, y1_ktest, y2_ktest), (x_utest, y1_utest, y2_utest) = split_dataset(dataset, personID, speedID)
(x_train_small, y1_train_small, y2_train_small), (x_train_large, y1_train_large, y2_train_large) = split_small_large_set(x_train, y1_train, y2_train, N_SELECT)
(x_ktest_small, y1_ktest_small, y2_ktest_small), (x_ktest_large, y1_ktest_large, y2_ktest_large) = split_small_large_set(x_ktest, y1_ktest, y2_ktest, N_SELECT)
(x_utest_small, y1_utest_small, y2_utest_small), (x_utest_large, y1_utest_large, y2_utest_large) = split_small_large_set(x_utest, y1_utest, y2_utest, 200)
y1 = torch.cat((y1_ktest_large, y1_utest_large+4))
x = torch.cat((x_ktest_large, x_utest_large))
y2 = torch.cat((y2_ktest_large, y2_utest_large+4))

model_eval_speed(model, x, y2)
model_eval_user(model, x, y1)